In [1]:
#### url:https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile
### webscraping_bus route details in red bus:Kerala

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd

# Initialize the WebDriver and open the website
driver = webdriver.Chrome()
driver.maximize_window()
driver.get('https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile')

# List to store the collected bus route data
bus_route_data = []

# Function to collect route details from the page
def route_details():
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "route_link")))
        route_elements = driver.find_elements(By.CLASS_NAME, "route")
        
        route_links = [route.get_attribute("href") for route in route_elements]
        route_names = [route.get_attribute("title").strip() for route in route_elements]
        
        # Collect the route details for this page and return them
        return [{'route_name': name, 'route_link': link} for name, link in zip(route_names, route_links)]
        
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Function to navigate through all the pages and scrape route details
def scrape_all_pages():
    all_routes = []
    try:
        # Loop through the first 5 pages (or adjust the range as needed)
        for page in range(1, 6):  # Assuming there are 5 pages
            if page > 1:
                # Wait for the pagination tab of the current page to be clickable
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                # Scroll to the pagination tab and click it
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the new page to load
            
            # Collect the route details after navigating to the page (either the first page or after pagination)
            all_routes.extend(route_details())

    except Exception as e:
        print(f"Error occurred while accessing page {page}: {str(e)}")
    return all_routes

# Function to scrape bus details for each route
def scrape_bus_details(route_name, route_link):
    try:
        driver.get(route_link)
        time.sleep(5)

        # Try to click the "View Buses" button if available
        try:
            view_buses = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses)
            time.sleep(5)
        except:
            print(f"No 'View Buses' button found for {route_link}")

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Extract data from the page
        bus_names = driver.find_elements(By.CSS_SELECTOR, "div.travels.lh-24.f-bold.d-color")
        bus_types = driver.find_elements(By.CSS_SELECTOR, "div.bus-type.f-12.m-top-16.l-color.evBus")
        departure_times = driver.find_elements(By.CSS_SELECTOR, "div.dp-time.f-19.d-color.f-bold")
        durations = driver.find_elements(By.CSS_SELECTOR, "div.dur.l-color.lh-24")
        reaching_times = driver.find_elements(By.CSS_SELECTOR, "div.bp-time.f-19.d-color.disp-Inline")
        star_ratings = driver.find_elements(By.CSS_SELECTOR, "div.lh-18.rating.rat-red")
        prices = driver.find_elements(By.CSS_SELECTOR, "div.fare.d-block")
        seat_availabilities = driver.find_elements(By.CSS_SELECTOR, "div.seat-left.m-top-16")
    
        bus_route_details = []
    
        # Loop through each bus and store its details
        for i in range(len(bus_names)):
            bus_detail = {
                "State": "Kerala",
                "Route Name": route_name,
                "Route Link": route_link,
                "Bus Name": bus_names[i].text,
                "Bus Type": bus_types[i].text if i < len(bus_types) else "N/A",
                "Departure Time": departure_times[i].text if i < len(departure_times) else "N/A",
                "Duration": durations[i].text if i < len(durations) else "N/A",
                "Reaching Time": reaching_times[i].text if i < len(reaching_times) else "N/A",
                "Rating": star_ratings[i].text if i < len(star_ratings) else "N/A",
                "Price": prices[i].text if i < len(prices) else "N/A",
                "Seats Available": seat_availabilities[i].text if i < len(seat_availabilities) else "N/A"
            }
            bus_route_details.append(bus_detail)
                     
        return bus_route_details
    except Exception as e:
        print(f"Error occurred while accessing {route_link}: {str(e)}")
        return []   

# Scrape all route details
routes = scrape_all_pages()

# For each route, scrape bus details and add to the bus_route_data list
for route in routes:
    bus_details = scrape_bus_details(route["route_name"], route["route_link"])
    if bus_details:
        bus_route_data.extend(bus_details)  # Add bus details to the bus_route_data list

# Convert bus_route_data to a pandas DataFrame
df = pd.DataFrame(bus_route_data)

# Save the DataFrame to a CSV file
df.to_csv('Kerala_bus_details.csv', index=False)

# Optionally, print the collected bus details
print("Data saved to Kerala_bus_details.csv")

# Close the driver after scraping
driver.quit()


Error occurred while accessing page 3: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7F0C96CF5+28821]
	(No symbol) [0x00007FF7F0C03880]
	(No symbol) [0x00007FF7F0AA578A]
	(No symbol) [0x00007FF7F0AF91BE]
	(No symbol) [0x00007FF7F0AF94AC]
	(No symbol) [0x00007FF7F0B42647]
	(No symbol) [0x00007FF7F0B1F33F]
	(No symbol) [0x00007FF7F0B3F412]
	(No symbol) [0x00007FF7F0B1F0A3]
	(No symbol) [0x00007FF7F0AEA778]
	(No symbol) [0x00007FF7F0AEB8E1]
	GetHandleVerifier [0x00007FF7F0FCFCED+3408013]
	GetHandleVerifier [0x00007FF7F0FE745F+3504127]
	GetHandleVerifier [0x00007FF7F0FDB63D+3455453]
	GetHandleVerifier [0x00007FF7F0D5BDFB+835995]
	(No symbol) [0x00007FF7F0C0EB9F]
	(No symbol) [0x00007FF7F0C0A854]
	(No symbol) [0x00007FF7F0C0A9ED]
	(No symbol) [0x00007FF7F0BFA1D9]
	BaseThreadInitThunk [0x00007FF8B0EFDBE7+23]
	RtlUserThreadStart [0x00007FF8B1E7FBEC+44]

Data saved to Kerala_bus_details.csv


In [3]:
df

,State,Route Name,Route Link,Bus Name,Bus Type,Departure Time,Duration,Reaching Time,Rating,Price,Seats Available
0,Kerala,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 553,Super Express Non AC Seater Air Bus (2+2),16:45,08h 55m,01:40,2.9,INR 640,N/A
1,Kerala,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 51,Swift Deluxe Non AC Air Bus (2+2),19:00,09h 30m,04:30,2.7,INR 640,N/A
2,Kerala,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 789,Super Fast Non AC Seater (2+3),20:00,07h 45m,03:45,N/A,INR 473,N/A
3,Kerala,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1585,Swift Deluxe Non AC Air Bus (2+2),20:40,09h 30m,06:10,N/A,INR 640,N/A
4,Kerala,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1586,Swift Deluxe Non AC Air Bus (2+2),21:30,09h 25m,06:55,N/A,INR 640,N/A
...,...,...,...,...,...,...,...,...,...,...,...
531,Kerala,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Ubc travels,A/C Seater/Sleeper (2+1),18:00,09h 00m,03:00,N/A,INR 900,N/A
532,Kerala,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Cochin Bus,Bharat Benz A/C Sleeper (2+1),19:50,06h 55m,02:45,N/A,INR 770,N/A
533,Kerala,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,A1 Travels,A/C Seater / Sleeper (2+1),23:59,07h 01m,07:00,N/A,INR 700,N/A
534,Kerala,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Kallada Travels (Suresh Kallada),A/C Sleeper (2+1),18:15,07h 10m,01:25,N/A,INR 1100,N/A
